In [1]:
%cd ..

D:\anomaly-detection


In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.signal import find_peaks

import cufflinks as cf
cf.go_offline(connected=True)

import bokeh.io
bokeh.io.output_notebook()

from src.visualization.visualize import ratios_plot

np.random.seed(42)

Loading BokehJS ...

# Preparing data

In [3]:
ratios = np.load('result/TEP_small-ratios.npy')

peaksP, _ = find_peaks(ratios, distance=100, prominence=1)
peaksN, _ = find_peaks(-ratios, distance=100, prominence=1.5)
peaks = sorted(np.concatenate((peaksP, peaksN), axis=0))

In [4]:
to_remove = []
for i, peak in enumerate(peaks):
    if np.isclose(ratios[peak + 100], ratios[peak - 100]) and np.isclose(
            ratios[peak + 100], ratios[peak]):
        to_remove.append(i)

for i in reversed(to_remove):
    peaks.pop(i)

peaks = np.array(peaks)

In [5]:
ratios_plot(ratios, peaks)

In [6]:
data = pd.read_csv('data/processed/tep_data.csv', index_col='Index')
print(f'Len of dataset: {data.shape[0]}')

Len of dataset: 12801


In [7]:
boundaries = [0, *list(peaks), ratios.shape[0]]

X = []
for left, right in zip(boundaries[:-1], boundaries[1:]):
    X.append(data.iloc[left:right].values)

In [8]:
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset

class Dataset:
    def __init__(self, X_tr, X_val):
        self.__dict__.update(locals())
        self.tslearn = to_time_series_dataset(self.X_tr + self.X_val)
        self.tr_labels = None
        self.val_labels = None
    
    def get(self):
        return self.X_tr, self.X_val
    
    def get_tslearn(self):
        return self.tslearn[:len(self.X_tr)], self.tslearn[len(self.X_tr):]

In [9]:
X_tr, X_te = train_test_split(X, train_size=0.75)
dataset = Dataset(X_tr, X_te)

# Clustering

In [12]:
def cluster_plot(labels):
    mx_row = np.max(np.bincount(labels))
    mx_col = np.max(labels) + 1

    plt.figure(figsize=(8 * mx_col, 6 * mx_row))
    for col in range(mx_col):
        idxs = np.where(labels == col)[0]
        for i in range(len(idxs)):
            plt.subplot(mx_row, mx_col, mx_col * i + col + 1)
            if i == 0:
                plt.title(str(col))
            plt.plot(data.iloc[boundaries[idxs[i]]:boundaries[idxs[i] + 1]])
            rng = list(range(boundaries[idxs[i]], boundaries[idxs[i] + 1]))
            step = len(rng) // 6
            plt.xticks(rng[::step])

## GlobalAlignmentKernelKMeans

In [11]:
from tslearn.clustering import GlobalAlignmentKernelKMeans
gak_km = GlobalAlignmentKernelKMeans(n_clusters=4)
labels_gak = gak_km.fit_predict(dataset.get_tslearn()[0])

C:\Anaconda3\envs\anom-det\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning:

The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.



In [ ]:
cluster_plot(labels_gak)

In [15]:
np.bincount(labels_gak)

array([ 8, 10,  3,  7], dtype=int64)

## TSKMeans with dtw

In [10]:
from tslearn.clustering import TimeSeriesKMeans, silhouette_score
km = TimeSeriesKMeans(n_clusters=4, metric="dtw")
labels_ts_km = km.fit_predict(dataset.get_tslearn()[0])
silhouette_score(dataset.get_tslearn()[0], labels_ts_km, metric="dtw")

C:\Anaconda3\envs\anom-det\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning:

The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.



0.08011042619146981

In [ ]:
cluster_plot(labels_ts_km)

In [20]:
np.bincount(labels_ts_km)

array([15, 13], dtype=int64)

# Compare

## Clusterization

In [15]:
from sklearn.model_selection import train_test_split
import keras

from src.models.anomalies import build_lstm, fit_generator
from src.features.build_features import rolling_window


prediction_len = 1
window_len = 128
batch_size = 32

In [16]:
def validation_generator(X, y, batch_size=64):
    assert len(X) == len(y)
    while True:
        for i in range(0, len(X), batch_size):
            yield np.array(X[i:i+batch_size]), np.array(y[i:i+batch_size])

In [17]:
class Clusterization:
    """
    Clusterization followed by classifier (Direct prediction clusterization is not working)
    """
    def __init__(self, cluster_model, classifier_model):
        self.cluster_model = cluster_model
        self.classifier_model = classifier_model
        
    def fit(self, X):
        self.train_labels = self.cluster_model.fit_predict(X)
        self.classifier_model.fit(X, self.train_labels)
        
    def predict(self, X):
        return self.classifier_model.predict(X)
        
    def fit_predict(self, X):
        self.fit(X)
        return self.predict(X)

In [18]:
from datetime import datetime
from copy import deepcopy
import os


class LSTMPerCluster:
    """
    Accepts variable length X
    """
    def __init__(self,
                 clusterization_model,
                 lstm_model_fn,
                 window_len,
                 name,
                 prediction_len,
                 logs_dir='logs',
                 models_dir='models'):
        self.__dict__.update(locals())
        self.models = []
        self.creation_time = datetime.now().strftime('%Y-%m-%d-%H-%M')

    def _rolling_windows_from_cluster(self, X_cluster):
        """
        Returns shuffled rolling windows from cluster samples
        """
        X_samples, y_samples = [], []
        prediction_len = self.prediction_len
        window_len = self.window_len

        for sample in X_cluster:
            X_samples.extend(
                rolling_window(sample[:-prediction_len], window_len))
            y_samples.extend(rolling_window(sample, prediction_len,
                                            window_len))

#         idxs = list(range(len(X_samples)))
#         np.random.shuffle(idxs)
#         X_samples = np.take(X_samples, idxs, axis=0)
#         y_samples = np.take(y_samples, idxs, axis=0)
        return np.array(X_samples), np.array(y_samples)

    def _cluster_rolling_windows(self, data, labels, label):
        X_cluster = np.take(data, np.nonzero(labels == label)[0], axis=0)

        return self._rolling_windows_from_cluster(X_cluster)

    def _add_tb_and_checkpoint_callbacks(self, params, cluster_n):
        params['callbacks'] = params.get('callbacks', [])

        directory = '/' + self.name + '_clusters_' + self.creation_time
        suffix = directory + '/' + str(cluster_n)

        params['callbacks'].append(
            keras.callbacks.TensorBoard(log_dir=self.logs_dir + suffix,
                                        histogram_freq=1,
                                        write_grads=True,
                                        update_freq='epoch'))

        if not os.path.exists(self.models_dir):
            os.mkdir(self.models_dir)

        if not os.path.exists(self.models_dir + directory):
            os.mkdir(self.models_dir + directory)

        params['callbacks'].append(
            keras.callbacks.ModelCheckpoint(filepath=self.models_dir + suffix +
                                            '.h5',
                                            save_best_only=True))

    def fit(self, dataset, lstm_fit_params, batch_size=64):
        """
        Accepts (X, y) pairs of different length
        """
        tr_labels = self.clusterization_model.fit_predict(
            dataset.get_tslearn()[0])
        val_labels = self.clusterization_model.predict(
            dataset.get_tslearn()[1])

        dataset.tr_labels = tr_labels
        dataset.val_labels = val_labels

        print(np.bincount(tr_labels))
        print(np.bincount(val_labels))

        for cluster in np.unique(tr_labels):
            X_tr_samples, y_tr_samples = self._cluster_rolling_windows(
                dataset.get()[0], tr_labels, cluster)

            if np.nonzero(val_labels == cluster)[0].shape[0] == 0:
                validation_data = None
                validation_steps = None
            else:
                X_val_samples, y_val_samples = self._cluster_rolling_windows(
                    dataset.get()[1], val_labels, cluster)

                #                 validation_data = validation_generator(X_val_samples,
                #                                                        y_val_samples,
                #                                                        batch_size)
                #                 validation_steps = (len(X_val_samples) + batch_size -
                #                                     1) // batch_size
                validation_data = (X_val_samples, y_val_samples)
                validation_steps = None
            
            model = self.lstm_model_fn()
            lstm_params_cpy = deepcopy(lstm_fit_params)
            self._add_tb_and_checkpoint_callbacks(lstm_params_cpy, cluster)

            model.fit_generator(
                generator=fit_generator(X_tr_samples, y_tr_samples,
                                        batch_size),
                steps_per_epoch=len(X_tr_samples) // batch_size,
                validation_data=validation_data,
                validation_steps=validation_steps,
                **lstm_params_cpy)

            self.models.append(model)
            keras.backend.clear_session()

In [30]:
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, TimeDistributed, Bidirectional, Reshape
from keras.regularizers import l2


def lstm_model(input_length,
               input_shape,
               prediction_len,
               lstm_layers_size,
               optimizer,
               reg_strength=0.01,
               dropout_coeff=0.1,
               **compile_attrs):
    """
    Builds lstm model with hidden layers of size `layers_size`.
    Returns values with shape (input_length, input_shape).
    Loss by default is `mae` and added to compile_attrs
    """
    model = Sequential()

    model.add(LSTM(lstm_layers_size[0],
                           return_sequences=True if len(lstm_layers_size) > 1 else False,
                           kernel_regularizer=l2(reg_strength),
                   activation='softsign',
                   input_shape=(input_length, input_shape)))

    for i, size in enumerate(lstm_layers_size[1:]):
        if 0 < dropout_coeff < 1:
            model.add(Dropout(dropout_coeff))

        model.add(
                LSTM(
                    size,
                    return_sequences=False if i == len(lstm_layers_size) - 2 else True,
                    kernel_regularizer=l2(reg_strength),
                    activation='softsign',
                ))
    
    model.add(Dense(prediction_len * input_shape, kernel_regularizer=l2(reg_strength)))
    model.add(Reshape((prediction_len, input_shape)))

    compile_attrs['optimizer'] = optimizer()
    compile_attrs['loss'] = compile_attrs.get('loss', 'mae')
    model.compile(**compile_attrs)

    return model

In [20]:
# from tslearn.svm import TimeSeriesSVC
# from tslearn.utils import to
# clf = TimeSeriesSVC(C=1.0, kernel="gak")
# clf.fit(X_tr_dataset.copy(), labels_gak)

# print(np.bincount(clf.predict(X_te_dataset.copy())))

In [27]:
config = dict(
    input_length=window_len,
    input_shape=data.shape[1],
    prediction_len=prediction_len,
    lstm_layers_size=[41],
    loss='mse',
    optimizer=lambda: keras.optimizers.Adam(lr=0.01),
    reg_strength=0.05,
)

fit_params = dict(
    epochs=20,
    verbose=1,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(patience=3, min_delta=0.005, factor=0.5),
        keras.callbacks.EarlyStopping(min_delta=0.01, patience=9),
    ]
)

In [33]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn.clustering import GlobalAlignmentKernelKMeans

np.random.seed(42)

cl_model_1 = Clusterization(
    GlobalAlignmentKernelKMeans(n_clusters=4, max_iter=100, n_jobs=2),
    KNeighborsTimeSeriesClassifier(n_neighbors=3, n_jobs=2)
)

model = LSTMPerCluster(cl_model_1, 
                       lambda: lstm_model(**config), 
                       window_len, 
                       'gak-knn-reduced-softsign',
                       prediction_len=prediction_len)

In [34]:
model.fit(dataset, fit_params, batch_size=32)

[8 9 4 7]
[3 3 1 3]
Epoch 1/20
31/31 [==============================] - 4s 121ms/step - loss: 1.9485 - val_loss: 0.9912
Epoch 2/20
31/31 [==============================] - 3s 98ms/step - loss: 0.7335 - val_loss: 1.3997
Epoch 3/20
31/31 [==============================] - 3s 102ms/step - loss: 0.8816 - val_loss: 1.0104
Epoch 4/20
31/31 [==============================] - 3s 98ms/step - loss: 0.6959 - val_loss: 0.8589
Epoch 5/20
31/31 [==============================] - 3s 96ms/step - loss: 0.6516 - val_loss: 0.8778
Epoch 6/20
31/31 [==============================] - 3s 96ms/step - loss: 0.6090 - val_loss: 0.8642
Epoch 7/20
31/31 [==============================] - 3s 95ms/step - loss: 0.8189 - val_loss: 1.0802
Epoch 8/20
31/31 [==============================] - 3s 96ms/step - loss: 0.7214 - val_loss: 0.9874
Epoch 9/20
31/31 [==============================] - 3s 96ms/step - loss: 0.6480 - val_loss: 0.9080
Epoch 10/20
31/31 [==============================] - 3s 99ms/step - loss: 0.6199 - val_

In [1]:
# evaluate_clusterization(
#     Clusterization(TimeSeriesKMeans(n_clusters=4, metric="dtw", max_iter=75, n_jobs=2),
#         KNeighborsTimeSeriesClassifier(n_neighbors=5, n_jobs=2)), 
#     'ts_km-knn')

In [18]:
# evaluate_clusterization(km, 'ts_km')

## Without clusterization

In [39]:
class OneCluster:
    def fit_predict(self, X):
        return np.zeros(X.shape[0], dtype=int)
    
    def predict(self, X):
        return self.fit_predict(X)

In [40]:
one_lstm_model = LSTMPerCluster(OneCluster(), lambda: lstm_model(**config), window_len, 'pure-lstm')

In [41]:
one_lstm_model.fit(dataset, fit_params)

[28]
[10]
Epoch 1/40
110/110 [==============================] - 9s 84ms/step - loss: 1.8261 - val_loss: 1.3723
Epoch 2/40
110/110 [==============================] - 8s 74ms/step - loss: 1.5316 - val_loss: 1.2171
Epoch 3/40
110/110 [==============================] - 8s 73ms/step - loss: 1.3812 - val_loss: 1.1136
Epoch 4/40
110/110 [==============================] - 8s 73ms/step - loss: 1.2490 - val_loss: 1.0357
Epoch 5/40
110/110 [==============================] - 8s 72ms/step - loss: 1.1642 - val_loss: 0.9733
Epoch 6/40
110/110 [==============================] - 8s 74ms/step - loss: 1.1040 - val_loss: 0.9230
Epoch 7/40
110/110 [==============================] - 8s 70ms/step - loss: 1.0440 - val_loss: 0.8793
Epoch 8/40
110/110 [==============================] - 8s 72ms/step - loss: 0.9878 - val_loss: 0.8419
Epoch 9/40
110/110 [==============================] - 8s 74ms/step - loss: 0.9451 - val_loss: 0.8093
Epoch 10/40
110/110 [==============================] - 8s 74ms/step - loss: 0.910

In [43]:
X_te_labels = cl_model_1.predict(dataset.get_tslearn()[1])

In [44]:
pure_lstm = one_lstm_model.models[0]
clusterization_lstms = model.models

In [47]:
# cl_model_1.cluster_model.to_hdf5('models/gak.h5')
# cl_model_1.classifier_model.to_hdf5('models/classif-over-gak.h5')

In [48]:
X_plot = np.take(dataset.get()[1], [np.argmax(X_te_labels == i) for i in range(4)], axis=0)

In [49]:

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

@interact(component=(0, 40))
def plot(component):
    plt.figure(figsize=(12, 12))
    for i in range(X_plot.shape[0]):
        sample = X_plot[i]
        plt.subplot(2, 2, i+1)
        cur_lstm = clusterization_lstms[i]
        
        roll_wind_cl = sample[:window_len].copy()[None, ...]
        roll_wind_full = sample[:window_len].copy()[None, ...]
        
        to_plot_cl = list(roll_wind_cl[0, :, component])
        to_plot_full = list(roll_wind_full[0, :, component])
        
        steps = (sample.shape[0] - window_len) // prediction_len
        for _ in range(steps):
            roll_wind_cl = cur_lstm.predict(roll_wind_cl)
            roll_wind_full = pure_lstm.predict(roll_wind_full)
            
            to_plot_cl.extend(roll_wind_cl[0, -prediction_len:, component])
            to_plot_full.extend(roll_wind_full[0, -prediction_len:, component])
        
        plt.plot(sample[:, component], label='real')
        plt.plot(to_plot_cl, label='cluster ' + str(i) + ' lstm')
        plt.plot(to_plot_full, label='full lstm')
        plt.legend()

interactive(children=(IntSlider(value=20, description='component', max=40), Output()), _dom_classes=('widget-i…

In [51]:
# from sklearn.metrics import mean_squared_error
# print(np.mean(((XY - ts_gak_lstm_prediction) ** 2).sum(axis=2).sum(axis=1)))

In [52]:
# print(np.mean(((XY - overall_lstm_prediction) ** 2).sum(axis=2).sum(axis=1)))